In [35]:
from Bio import Entrez, SeqIO

In [36]:
# Hàm để tìm kiếm các peptide dựa trên từ khóa và giới hạn kích thước
def search_peptides(term, retmax=5000):
    handle = Entrez.esearch(
        db="protein",
        term=term,
        retmax=retmax,
        retmode="xml"
    )
    results = Entrez.read(handle)
    handle.close()
    return results["IdList"]

In [37]:
# Hàm để lấy trình tự từ NCBI dựa trên ID
def fetch_sequences(id_list):
    handle = Entrez.efetch(
        db="protein",
        id=id_list,
        rettype="fasta",
        retmode="text"
    )
    sequences = list(SeqIO.parse(handle, "fasta"))
    handle.close()
    return sequences

In [38]:
# Hàm để lọc trình tự theo độ dài từ 10 đến 50 amino acid
def filter_sequences(sequences, min_len=10, max_len=50):
    return [seq for seq in sequences if min_len <= len(seq.seq) <= max_len]

In [39]:
# Lưu trình tự vào file với gán nhãn "AMP" hoặc "nAMP"
def save_sequences_to_file(sequences, label, filename = "10_50.fasta"):
    with open(filename, "a") as output_handle:
        for seq in sequences:
            seq.id = f"{seq.id}" 
            seq.description = f"{label}"
            SeqIO.write(seq, output_handle, "fasta")

In [40]:
# Tìm kiếm và xử lý AMP (antimicrobial peptides)
amp_ids = search_peptides("antimicrobial peptide AND 10:50[Sequence Length]")
amp_sequences = fetch_sequences(amp_ids)
filtered_amp_sequences = filter_sequences(amp_sequences)

save_sequences_to_file(filtered_amp_sequences, "AMP")

In [41]:
# Tìm kiếm và xử lý nAMP (non-antimicrobial peptides)
namp_ids = search_peptides("NOT antimicrobial peptide AND 10:50[Sequence Length]")
namp_sequences = fetch_sequences(namp_ids)
filtered_namp_sequences = filter_sequences(namp_sequences)

save_sequences_to_file(filtered_namp_sequences, "nAMP")